In [30]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize

%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Number 1

In [33]:
#load in via pandas
wbp = pd.read_json('world_bank_projects.json', orient = 'table')
wbpl = pd.read_json('world_bank_projects_less.json', orient = 'table')

wbpl.head()

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{u'$oid': u'52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{u'Percent': 100, u'Name': u'Education for all'}","[{u'code': u'65', u'name': u'Education for all'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{u'$oid': u'52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{u'Percent': 30, u'Name': u'Other economic man...","[{u'code': u'24', u'name': u'Other economic ma...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


In [34]:
#bring together the two Df's
wbp2 = wbp.append(wbpl)
wbp2[['mjtheme','id']]



#I used .info(), .describe(), and .head() but removed them for brevity/polish

,mjtheme,id
0,[Human development],P129828
1,"[Economic management, Social protection and ri...",P144674
2,"[Trade and integration, Public sector governan...",P145310
3,"[Social dev/gender/inclusion, Social dev/gende...",P144665
4,"[Trade and integration, Financial and private ...",P144933
5,[Social protection and risk management],P146161
6,"[Public sector governance, Financial and priva...",P121185
7,[Environment and natural resources management],P127033
8,[Rural development],P130164
9,"[Public sector governance, Public sector gover...",P130903


In [37]:
#groupby country code, then find the size of each amalgamation, then sort
wbp2.groupby('countryname').size().sort_values(ascending = False).head(10)

countryname
Republic of Indonesia              19
People's Republic of China         19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
People's Republic of Bangladesh    12
Nepal                              12
Kingdom of Morocco                 12
Africa                             11
Republic of Mozambique             11
dtype: int64

# Number 2

In [40]:
#after near infinite hours, decided to bring in using json and normalize
with open('world_bank_projects.json') as j:
    f1 = json.load(j)
df1 = json_normalize(f1, 'mjtheme_namecode',['id'])


with open('world_bank_projects_less.json') as j:
    f2 = json.load(j)
#normalize using the column we need to measure (ID as well so we can merge on it later)
df2 = json_normalize(f2, 'mjtheme_namecode',['id'])
#concatenate them together
df = df1.append(df2)


In [41]:
#groupby code and sort to find which codes are most used
df.groupby('code').size().sort_values(ascending = False)

code
11    251
10    216
8     211
2     199
6     169
4     146
7     130
5      77
9      50
1      39
3      15
dtype: int64

# Number 3


My plan was to bring in the names from the mjtheme column and join on ID with the mjtheme_namecode-containing table, but it is not working. I tried both bringing the json in and normalizing, and pd.read_json and splitting the lists that are contained within the various cells. I spent a good 6 hours trying various approaches, only to met with various errors that I could not figure out. Stackoverflow and I got really close, as did the forums on springboard. One of the mentors posted similar code to that shown below as a help for people in a similar situation to me.

In [42]:
#Use pandas to bring in a messy version with all the information
df0 = pd.read_json('world_bank_projects.json')
df1 = pd.read_json('world_bank_projects_less.json')

#bring in a JSON that can be refined to have the information we are particularly interested in
with open('world_bank_projects.json') as f:    
    raw = json.load(f)
with open('world_bank_projects_less.json') as f:    
    raw1 = json.load(f)

#We know ID will help us join the correct names later on
df_themes0 = json_normalize(raw, 'mjtheme_namecode', ['id'])
df_themes1 = json_normalize(raw1, 'mjtheme_namecode', ['id'])

df_themes = df_themes0.append(df_themes1)
df = df0.append(df1)

# Here we see a number of projects with multiple codes, meaning the project spans many themes
df_themes.groupby('id').code.nunique().sort_values(ascending=False).head(10)

# Addtionally we have the problems of a missing theme name. I was able to get this far, but then I was quickly derailed
print 'Top 10 Major World Bank Project Themes:'
print (df_themes[df_themes != ''].name.value_counts().head(10))

# The holy grail (at least in my tired opinion)
df_themes_name_to_code = df_themes.groupby('name').code.max().sort_values(ascending=False)

# drop the missing name rows
df_themes_name_to_code = df_themes_name_to_code[df_themes_name_to_code.index != '']

# convert to df
df_themes_name_to_code = pd.DataFrame(df_themes_name_to_code,columns=['code'])

# pull name into a column
df_themes_name_to_code['name_clean'] = df_themes_name_to_code.index

# set code to be the index
df_themes_code_to_name = df_themes_name_to_code.set_index(['code'])


# now merge on the name based on the code for the missing projects

df_themes_clean = df_themes.merge(df_themes_code_to_name,how='outer',left_on=['code'],right_index=True)

df_final = df_themes_clean.drop('name')

#drop is no cooperating?

print(df_final[df_final != ''].dropna())





Top 10 Major World Bank Project Themes:
Environment and natural resources management    223
Rural development                               202
Human development                               198
Public sector governance                        184
Social protection and risk management           159
Financial and private sector development        130
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Economic management                              34
Name: name, dtype: int64
     code               name       id         name_clean
0       8  Human development  P129828  Human development
41      8  Human development  P132616  Human development
43      8  Human development  P146271  Human development
44      8  Human development  P146271  Human development
47      8  Human development  P146271  Human development
57      8  Human development  P146109  Human development
58      8  Human